In [1]:
# import the necessary libraries
import cv2
import numpy as np
import pandas as pd
import h5py
import matplotlib.pyplot as plt
%matplotlib inline

pd.set_option('display.max_rows',1000)
pd.set_option('display.max_columns',1000)

from keras.layers import Conv2D, MaxPooling2D, AveragePooling2D, Activation,\
BatchNormalization, Dense, Flatten, Input, add
from keras.optimizers import SGD
from keras.regularizers import l2
from keras.models import Model
from keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint, TensorBoard
from keras.preprocessing.image import ImageDataGenerator
from keras.datasets import cifar10
import keras.backend as K

from sklearn.preprocessing import LabelBinarizer

C:\Anaconda\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


Couldn't import dot_parser, loading of dot files will not be possible.


In [2]:
def residual_module(inputs, num_filters, stride, reduce=False,
                    reg_str=0.0001, bn_eps=2e-5, bn_mom=0.9):
    '''
    pre-activation residual module
    ---
    Args:
        inputs: tuple, image input with volume of (height, width, depth)
        num_filters: integer, number of convolving kernels
        strides: tuple, number of strides for convolving filter to take
        reduce: boolean, whether to reduce spatial dimension of image volume
        reg_str: float, regularization strength
        bn_eps: float, batch normalization epsilon
        bn_mom: float, batch normalization momentum

    Returns:
        x:
    '''
    shortcut = inputs
    # first block of ResNet module - 1 x 1 CONV
    bn1 = BatchNormalization(epsilon=bn_eps,
                             momentum=bn_mom)(inputs)
    act1 = Activation('relu')(bn1)
    conv1 = Conv2D(filters=int(num_filters*0.25), kernel_size=(1, 1),
                   use_bias=False, kernel_regularizer=l2(reg_str))(act1)

    # second block of ResNet module - 3 x 3 CONV
    bn2 = BatchNormalization(epsilon=bn_eps,
                             momentum=bn_mom)(conv1)
    act2 = Activation('relu')(bn2)
    conv2 = Conv2D(filters=int(num_filters*0.25), kernel_size=(3, 3),
                   strides=stride, padding='same', use_bias=False,
                   kernel_regularizer=l2(reg_str))(act2)

    # third and final block of ResNet module - 1 x 1 CONV
    bn3 = BatchNormalization(epsilon=bn_eps,
                             momentum=bn_mom)(conv2)
    act3 = Activation('relu')(bn3)
    conv3 = Conv2D(filters=num_filters, kernel_size=(1, 1), use_bias=False,
                   kernel_regularizer=l2(reg_str))(act3)

    # if we are to reduce the spatial size, apply a CONV layer to the shortcut
    if reduce:
        shortcut = Conv2D(filters=num_filters, kernel_size=(1, 1),
                          strides=stride, use_bias=False,
                          kernel_regularizer=l2(reg_str))(act1)

    # add together the shortcut and the final CONV
    x = add([conv3, shortcut])

    # return the addition as the output of the ResNet module
    return x

In [3]:
# global parameter settings
reg_str = 0.0005
bn_eps = 2e-5
bn_mom = 0.9
num_classes = 10
max_epochs = 50
batch_size = 128
learning_rate = 0.1

stages = [9,9,9] # number of blocks within ResNet (i.e. 3 blocks with 9 modules each)
filters = [64,64,128,256] # number of convolving kernels to use: 
                            # 64 - input structure
                            # 64 - first ResNet block
                            # 128 - second ResNet block
                            # 256 - third ResNet block

In [4]:
input = Input(shape=(32, 32, 3))
input.shape

TensorShape([Dimension(None), Dimension(32), Dimension(32), Dimension(3)])

In [5]:
# input structure
x = BatchNormalization()(input)
x = Conv2D(filters=filters[0],kernel_size=3,strides=(1,1),padding='same',use_bias=False,
          kernel_regularizer=l2(reg_str))(x)
print(x.shape)

(?, 32, 32, 64)


In [6]:
# resnet structure
for i in range(0,len(stages)):
    stride = (1,1) if i == 0 else (2,2)
    x = residual_module(inputs=x,num_filters=filters[i+1],stride=stride,reduce=True,
                        bn_eps=bn_eps,bn_mom=bn_mom)
    
    for j in range(0,stages[i]-1):
        x = residual_module(inputs=x,num_filters=filters[i+1],stride=(1,1),
                            bn_eps=bn_eps,bn_mom=bn_mom)
print(x.shape)

(?, 8, 8, 256)


In [7]:
# output structure
x = BatchNormalization(epsilon=bn_eps,momentum=bn_mom)(x)
x = Activation('relu')(x)
x = AveragePooling2D(pool_size=(8,8))(x)
print(x.shape)

x = Flatten()(x)
x = Dense(num_classes,kernel_regularizer=l2(reg_str))(x)
x = Activation('softmax')(x)
print(x.shape)
model = Model(inputs=input,outputs=x)
model.summary()

(?, 1, 1, 256)
(?, 10)
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 32, 32, 3)    0                                            
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 32, 32, 3)    12          input_1[0][0]                    
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 32, 32, 64)   1728        batch_normalization_1[0][0]      
__________________________________________________________________________________________________
batch_normalization_2 (BatchNor (None, 32, 32, 64)   256         conv2d_1[0][0]                   
______________________________________________________________________________________

In [8]:
# cifar10 dataset
((xtrain,ytrain), (xtest,ytest)) = cifar10.load_data()
xtrain = xtrain.astype(np.float32)
xtest = xtest.astype(np.float32)

# normalizing
xtrain = xtrain / 255.0
xtest = xtest / 255.0

# one hot encode
lb = LabelBinarizer()
ytrain = lb.fit_transform(ytrain)
ytest = lb.transform(ytest)

In [9]:
# callbacks
callbacks = [EarlyStopping(monitor='val_acc',
                          patience=8,
                          verbose=1,
                          min_delta=1e-4,
                          mode='max'),
            ReduceLROnPlateau(monitor='val_acc',
                             factor=0.1,
                             patience=4,
                             verbose=1,
                             epsilon=1e-4,
                             mode='max'),
            ModelCheckpoint(monitor='val_acc',
                           filepath='weights/resnet56_1_best_weights.hdf5',
                           save_best_only=True,
                           save_weights_only=True,
                           verbose=1,
                           mode='max'),
            TensorBoard(log_dir='logs')]

C:\Anaconda\lib\site-packages\keras\callbacks.py:1065: UserWarning: `epsilon` argument is deprecated and will be removed, use `min_delta` instead.
  warnings.warn('`epsilon` argument is deprecated and '


In [10]:
# training
model.compile(loss='categorical_crossentropy',
             optimizer=SGD(learning_rate),
             metrics=['accuracy'])

datagen = ImageDataGenerator(rotation_range=10,
                            width_shift_range=5./32,
                            height_shift_range=5./32,
                            horizontal_flip=True)

datagen.fit(xtrain)

history = model.fit_generator(generator=datagen.flow(xtrain,ytrain,batch_size=batch_size),
                             steps_per_epoch=np.ceil(float(len(xtrain)) / float(batch_size)),
                             epochs=max_epochs,
                             verbose=1,
                             callbacks=callbacks,
                             validation_data=(xtest,ytest),
                             validation_steps=np.ceil(float(len(xtest)) / float(batch_size)))

Epoch 1/50
391/391 [==============================] - 187s 478ms/step - loss: 2.2045 - acc: 0.3495 - val_loss: 2.0529 - val_acc: 0.4103

Epoch 00001: val_acc improved from -inf to 0.41030, saving model to weights/resnet56_1_best_weights.hdf5
Epoch 2/50
391/391 [==============================] - 186s 475ms/step - loss: 1.8790 - acc: 0.4828 - val_loss: 1.8356 - val_acc: 0.5100

Epoch 00002: val_acc improved from 0.41030 to 0.51000, saving model to weights/resnet56_1_best_weights.hdf5
Epoch 3/50
391/391 [==============================] - 194s 495ms/step - loss: 1.7062 - acc: 0.5504 - val_loss: 1.7676 - val_acc: 0.5403

Epoch 00003: val_acc improved from 0.51000 to 0.54030, saving model to weights/resnet56_1_best_weights.hdf5
Epoch 4/50
391/391 [==============================] - 182s 466ms/step - loss: 1.5677 - acc: 0.6030 - val_loss: 1.7682 - val_acc: 0.5663

Epoch 00004: val_acc improved from 0.54030 to 0.56630, saving model to weights/resnet56_1_best_weights.hdf5
Epoch 5/50
391/391 [===

391/391 [==============================] - 179s 459ms/step - loss: 0.5791 - acc: 0.9258 - val_loss: 0.7562 - val_acc: 0.8747

Epoch 00038: val_acc did not improve from 0.87710
Epoch 39/50
391/391 [==============================] - 179s 458ms/step - loss: 0.5747 - acc: 0.9267 - val_loss: 0.7464 - val_acc: 0.8772

Epoch 00039: val_acc improved from 0.87710 to 0.87720, saving model to weights/resnet56_1_best_weights.hdf5
Epoch 40/50
391/391 [==============================] - 179s 458ms/step - loss: 0.5739 - acc: 0.9263 - val_loss: 0.7528 - val_acc: 0.8745

Epoch 00040: val_acc did not improve from 0.87720
Epoch 41/50
391/391 [==============================] - 179s 458ms/step - loss: 0.5737 - acc: 0.9264 - val_loss: 0.7600 - val_acc: 0.8734

Epoch 00041: val_acc did not improve from 0.87720
Epoch 42/50
391/391 [==============================] - 180s 460ms/step - loss: 0.5700 - acc: 0.9269 - val_loss: 0.7503 - val_acc: 0.8767

Epoch 00042: val_acc did not improve from 0.87720
Epoch 43/50
39